In [47]:
import pandas as pd
import os
from functools import reduce

In [132]:
# read csv files as df

# Voting record of each county in California
df_ca_poli = pd.read_csv(r'C:\Users\ethan\OneDrive\Desktop\CA_U.SHouseofRepresentatives.csv', thousands=',')
df_ca_rev = pd.read_csv(r'C:\Users\ethan\OneDrive\Desktop\TotalRevenueCA2017.csv', thousands=',')

# 
df_ca_rev_clean = df_ca_rev
df_ca_rev_clean["Score"] = df_ca_rev_clean["Percentage"] * 100

# 
ca_key_vals = df_ca_poli.county.unique()
df_ca_poli_clean = pd.DataFrame(columns=["County", "Republican", "Democrat", "Other"])
df_ca_poli_clean["County"] = ca_key_vals

# SNAP
snap_ca = pd.read_csv(r'C:\Users\ethan\OneDrive\Desktop\SNAPDataCA.csv', thousands=',')
snap_ca.columns = ["County", "SNAP_Percentage", "Total_Pop"]
snap_ca['County'] = snap_ca.County.str.replace('County, CA' , '')
snap_ca['SNAP_Percentage'] = snap_ca['SNAP_Percentage'].str.rstrip('%').astype('float') / 100.0

# Income Bracket
income_brackets_ca = pd.read_csv(r'C:\Users\ethan\OneDrive\Desktop\IncomeBracketsCA.csv', thousands=',')
income_brackets_ca.columns = ["County", "one_three", "three_eight_four", "Total_Pop"]
income_brackets_ca['County'] = income_brackets_ca.County.str.replace('County, CA' , '')
income_brackets_ca['one_three'] = income_brackets_ca['one_three'].str.rstrip('%').astype('float') / 100.0
income_brackets_ca['three_eight_four'] = income_brackets_ca['three_eight_four'].str.rstrip('%').astype('float') / 100.0

dataframes_to_clean = [df_ca_poli_clean, df_ca_rev, snap_ca, income_brackets_ca]
# for df in dataframes_to_clean:
#     df.columns = df.columns.str.replace(' ', '')
#     df = df.sort_values('County').reset_index(drop=True)
# #     print(df)

def clean_up_df(df):
    df.columns = df.columns.str.replace(' ', '')
    df = df.sort_values('County').reset_index(drop=True)
    return df

df_ca_poli_clean = clean_up_df(df_ca_poli_clean)
df_ca_rev_clean = clean_up_df(df_ca_rev_clean)
snap_ca = clean_up_df(snap_ca)
income_brackets_ca = clean_up_df(income_brackets_ca)

# df_ca_poli_clean.columns = df_ca_poli_clean.columns.str.replace(' ', '')
# df_ca_poli_clean = df_ca_poli_clean.sort_values('County').reset_index(drop=True)

# print(df_ca_poli_clean)

In [133]:
print(df_ca_poli_clean)

             County Republican Democrat Other
0           Alameda        NaN      NaN   NaN
1            Alpine        NaN      NaN   NaN
2            Amador        NaN      NaN   NaN
3             Butte        NaN      NaN   NaN
4         Calaveras        NaN      NaN   NaN
5            Colusa        NaN      NaN   NaN
6      Contra Costa        NaN      NaN   NaN
7         Del Norte        NaN      NaN   NaN
8      Districtwide        NaN      NaN   NaN
9         El Dorado        NaN      NaN   NaN
10           Fresno        NaN      NaN   NaN
11            Glenn        NaN      NaN   NaN
12         Humboldt        NaN      NaN   NaN
13         Imperial        NaN      NaN   NaN
14             Inyo        NaN      NaN   NaN
15             Kern        NaN      NaN   NaN
16            Kings        NaN      NaN   NaN
17             Lake        NaN      NaN   NaN
18           Lassen        NaN      NaN   NaN
19      Los Angeles        NaN      NaN   NaN
20           Madera        NaN    

In [134]:
print(income_brackets_ca)

              County  one_three  three_eight_four  Total_Pop
0           Alameda      0.0766            0.0406     379405
1            Alpine      0.0485            0.0388        206
2            Amador      0.0717            0.0324       9472
3             Butte      0.1227            0.0539      51488
4         Calaveras      0.0717            0.0358      11896
5            Colusa      0.1369            0.0872       5309
6      Contra Costa      0.0755            0.0376     279729
7         Del Norte      0.1737            0.0542       5735
8         El Dorado      0.0558            0.0288      48175
9            Fresno      0.2246            0.0744     219884
10            Glenn      0.1538            0.0770       7193
11         Humboldt      0.1293            0.0701      30401
12         Imperial      0.2063            0.0704      35069
13             Inyo      0.0753            0.0423       4354
14             Kern      0.2075            0.0830     198056
15            Kings     

In [69]:
# # copy_snap_ca = snap_ca[0:1].copy()
# # snap_ca['County'] = copy_snap_ca['County'].str.rstrip(' County, CA')
# snap_ca['County'] = snap_ca.County.str.replace('County, CA' , '')
# snap_ca['SNAP_Percentage'] = snap_ca['SNAP_Percentage'].str.rstrip('%').astype('float') / 100.0
# print(snap_ca)

In [135]:
# GOAL: Process County Level Data to Get Political affiliations of county

# Get Key Values and Set Up New Data Frame
# ca_key_vals = df_ca_poli.county.unique()
# df_ca_poli_clean = pd.DataFrame(columns=["County", "Republican", "Democrat", "Other"])
# df_ca_poli_clean["County"] = ca_key_vals
# print(df_ca_poli_clean)

# Calculate percentage
find_affiliation_tot = lambda df, affil, p_value, v_value : df.loc[df[value] == affil][v_value].sum()
for county in ca_key_vals:
    county_df = df_ca_poli.loc[df_ca_poli["county"] == county]
    county_total_votes = county_df["votes"].sum()
    dem_votes = county_df.loc[county_df["party"] == "DEM"]["votes"].sum()
    rep_votes = county_df.loc[county_df["party"] == "REP"]["votes"].sum()
    other_votes = county_df.loc[county_df["party"] == "NPP"]["votes"].sum()

    df_ca_poli_clean.loc[df_ca_poli_clean["County"] == county, ["Republican"]] = (rep_votes/county_total_votes)*100
    df_ca_poli_clean.loc[df_ca_poli_clean["County"] == county, ["Democrat"]] = (dem_votes/county_total_votes)*100
    df_ca_poli_clean.loc[df_ca_poli_clean["County"] == county, ["Other"]] = (other_votes/county_total_votes)*100

df_ca_poli_clean = df_ca_poli_clean[df_ca_poli_clean.County != "Districtwide"]
df_ca_poli_clean["Score"] = df_ca_poli_clean["Democrat"]
print(df_ca_poli_clean)

# Remove CountyWide

             County Republican Democrat      Other    Score
0           Alameda    13.5254  84.9187    1.31708  84.9187
1            Alpine    36.1413  63.8587          0  63.8587
2            Amador     63.405   36.595          0   36.595
3             Butte    50.2101  48.7744          0  48.7744
4         Calaveras    61.7042  38.2958          0  38.2958
5            Colusa    56.2917  43.7083          0  43.7083
6      Contra Costa    25.3324  69.4382    4.01434  69.4382
7         Del Norte    49.5748  50.4252          0  50.4252
9         El Dorado    58.8502  41.1498          0  41.1498
10           Fresno    53.5882  45.0906   0.787114  45.0906
11            Glenn    63.6746  36.3254          0  36.3254
12         Humboldt    30.3048  69.6952          0  69.6952
13         Imperial     38.868  57.1442    3.98787  57.1442
14             Inyo    55.2075  44.7925          0  44.7925
15             Kern    68.7732  29.5707    1.65611  29.5707
16            Kings    72.2987  27.7013 

In [71]:
# Goal: Process Population Data
# df_ca_rev_clean = df_ca_rev
# df_ca_rev_clean["Score"] = df_ca_rev_clean["Percentage"] * 100
# print(df_ca_rev_clean)

             County      Revenue  Percentage  Score
0       Los Angeles  23422437171      0.3003  30.03
1       Santa Clara   5625107619      0.0721   7.21
2            Orange   4412670251      0.0566   5.66
3         San Diego   4389789349      0.0563   5.63
4         Riverside   4343263894      0.0557   5.57
5    San Bernardino   4141319948      0.0531   5.31
6        Sacramento   3355290000      0.0430   4.30
7      Contra Costa   3072465406      0.0394   3.94
8           Alameda   2949248612      0.0378   3.78
9           Ventura   1900770086      0.0244   2.44
10             Kern   1834349504      0.0235   2.35
11        San Mateo   1732198692      0.0222   2.22
12           Fresno   1683934000      0.0216   2.16
13      San Joaquin   1439251199      0.0185   1.85
14         Monterey   1085600914      0.0139   1.39
15           Sonoma   1070620681      0.0137   1.37
16       Stanislaus   1039155522      0.0133   1.33
17    Santa Barbara    880935692      0.0113   1.13
18          

In [136]:
# SNAP Data
snap_ca["Score"] = snap_ca["SNAP_Percentage"] * 100
print(snap_ca)

# Poverty Data for population

              County  SNAP_Percentage  Total_Pop  Score
0           Alameda            0.0680     379405   6.80
1            Alpine            0.0874        206   8.74
2            Amador            0.0723       9472   7.23
3             Butte            0.1046      51488  10.46
4         Calaveras            0.0615      11896   6.15
5            Colusa            0.1192       5309  11.92
6      Contra Costa            0.0633     279729   6.33
7         Del Norte            0.1313       5735  13.13
8         El Dorado            0.0433      48175   4.33
9            Fresno            0.1962     219884  19.62
10            Glenn            0.0817       7193   8.17
11         Humboldt            0.1166      30401  11.66
12         Imperial            0.1943      35069  19.43
13             Inyo            0.0783       4354   7.83
14             Kern            0.1510     198056  15.10
15            Kings            0.1512      32205  15.12
16             Lake            0.0901      16029

In [137]:
income_brackets_ca["Score"] = (income_brackets_ca["one_three"] + income_brackets_ca["three_eight_four"]) * 100
print(income_brackets_ca)

              County  one_three  three_eight_four  Total_Pop  Score
0           Alameda      0.0766            0.0406     379405  11.72
1            Alpine      0.0485            0.0388        206   8.73
2            Amador      0.0717            0.0324       9472  10.41
3             Butte      0.1227            0.0539      51488  17.66
4         Calaveras      0.0717            0.0358      11896  10.75
5            Colusa      0.1369            0.0872       5309  22.41
6      Contra Costa      0.0755            0.0376     279729  11.31
7         Del Norte      0.1737            0.0542       5735  22.79
8         El Dorado      0.0558            0.0288      48175   8.46
9            Fresno      0.2246            0.0744     219884  29.90
10            Glenn      0.1538            0.0770       7193  23.08
11         Humboldt      0.1293            0.0701      30401  19.94
12         Imperial      0.2063            0.0704      35069  27.67
13             Inyo      0.0753            0.042

In [75]:
# Calculate the total score
# data_frames = [df_ca_poli_clean, df_ca_rev_clean, snap_ca, income_brackets_ca]

# df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['County'],
#                                             how='outer'), data_frames)

# print(df_merged)

               County Republican Democrat     Other  Score_x       Revenue  \
0               Butte    50.2101  48.7744         0  48.7744  4.051801e+08   
1               Glenn    63.6746  36.3254         0  36.3254  7.471302e+07   
2              Lassen    74.5867  24.8947         0  24.8947  6.478900e+07   
3               Modoc    69.5936  29.9393         0  29.9393  3.086332e+07   
4              Nevada    47.3238  51.6885         0  51.6885  1.638629e+08   
5              Placer    57.7438  42.1241         0  42.1241  6.585756e+08   
6              Plumas    56.7357  42.2953         0  42.2953  5.646060e+07   
7              Shasta    67.6152  31.5077         0  31.5077  3.252912e+08   
8              Sierra    61.1111  37.6369         0  37.6369  1.924739e+07   
9            Siskiyou    56.9112  39.6675         0  39.6675  9.610459e+07   
10             Tehama    70.2784  29.0256         0  29.0256  1.297442e+08   
11          Del Norte    49.5748  50.4252         0  50.4252  6.

In [143]:
# Apply metric weights
data_frames = [df_ca_poli_clean, df_ca_rev_clean, snap_ca, income_brackets_ca]
# METRICS
metrics = [.5, 1.5, 1.25, 1.25]
print(df_ca_rev_clean)
for df, metric in zip(data_frames, metrics):
    df["Score"] = df["Score"].apply(lambda x: x*metric)

print(df_ca_rev_clean)

             County      Revenue  Percentage   Score
0           Alameda   2949248612      0.0378   5.670
1            Alpine     18935581      0.0002   0.030
2            Amador     73472682      0.0009   0.135
3             Butte    405180116      0.0052   0.780
4         Calaveras    103847016      0.0013   0.195
5            Colusa     56417543      0.0007   0.105
6      Contra Costa   3072465406      0.0394   5.910
7         Del Norte     68538964      0.0009   0.135
8         El Dorado    320597586      0.0041   0.615
9            Fresno   1683934000      0.0216   3.240
10            Glenn     74713016      0.0010   0.150
11         Humboldt    307219596      0.0039   0.585
12         Imperial    438301291      0.0056   0.840
13             Inyo     75895392      0.0010   0.150
14             Kern   1834349504      0.0235   3.525
15            Kings    268613402      0.0034   0.510
16             Lake    141621306      0.0018   0.270
17           Lassen     64789003      0.0008  

In [140]:
# snap_ca["Score"] = pd.to_numeric(snap_ca["Score"])
# income_brackets_ca["Score"] = pd.to_numeric(income_brackets_ca["Score"])
# test = pd.merge(snap_ca, income_brackets_ca, on="County")
data_frames = [df_ca_poli_clean, df_ca_rev_clean, snap_ca, income_brackets_ca]
bigdata = pd.concat(data_frames, ignore_index=True)
print(bigdata)

               County Democrat     Other  Percentage Republican  Revenue  \
0             Alameda  84.9187   1.31708         NaN    13.5254      NaN   
1              Alpine  63.8587         0         NaN    36.1413      NaN   
2              Amador   36.595         0         NaN     63.405      NaN   
3               Butte  48.7744         0         NaN    50.2101      NaN   
4           Calaveras  38.2958         0         NaN    61.7042      NaN   
5              Colusa  43.7083         0         NaN    56.2917      NaN   
6        Contra Costa  69.4382   4.01434         NaN    25.3324      NaN   
7           Del Norte  50.4252         0         NaN    49.5748      NaN   
8           El Dorado  41.1498         0         NaN    58.8502      NaN   
9              Fresno  45.0906  0.787114         NaN    53.5882      NaN   
10              Glenn  36.3254         0         NaN    63.6746      NaN   
11           Humboldt  69.6952         0         NaN    30.3048      NaN   
12          

C:\Users\ethan\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [141]:
top_ca_counties = bigdata.sort_values('Score',ascending = False).head(5)

In [144]:
print(top_ca_counties)

           County Democrat    Other  Percentage Republican       Revenue  \
76    Los Angeles      NaN      NaN      0.3003        NaN  2.342244e+10   
37  San Francisco  85.7948  1.33562         NaN     10.982           NaN   
0         Alameda  84.9187  1.31708         NaN    13.5254           NaN   
20          Marin    84.38        0         NaN      15.62           NaN   
43     Santa Cruz  83.9214  9.88408         NaN    6.19452           NaN   

    SNAP_Percentage      Score  Total_Pop  one_three  three_eight_four  
76              NaN  45.045000        NaN        NaN               NaN  
37              NaN  42.897420        NaN        NaN               NaN  
0               NaN  42.459336        NaN        NaN               NaN  
20              NaN  42.190010        NaN        NaN               NaN  
43              NaN  41.960701        NaN        NaN               NaN  
